# 分類問題とは？

* 何かをグループに分ける処理　主に「教師あり学習」の場合を指す
* 一方で、何か大小に意味のある数値を予測する処理を、回帰問題と呼ぶ
* ベクトル空間モデルを利用した場合、文書などのアイテムは多次元空間の点として表すことができる　素性数が2なら二次元空間上の点
* 決定関数を用いて分類する境界線を表す
* 汎用性の高い学習データを用いて学習を行う必要がある。
* 開発データを用いて、未学習と過学習のバランスをとる
* sklearn.model_selection.train_test_splitモジュールで、データを分割することができる

# NaiveBayes

* 分類アルゴリズムで最も単純な方法
* ベイズの定理：$ P(X)P(Y|X)=P(Y)P(X|Y) $
* $ P(C|d)=P(C|w_1,w_2,...,w_n), 文書d, i番目の単語:w_i, 分類先のクラス $
* $ ある文書d=w_1,...,w_nがクラスCに属する確率 $
* $ P(d)P(C|d)=P(C)P(d|C)より、P(C|d)=\frac{P(C)P(d|C)}{P(d)} $ ...
* $ P(C|d) $：文書内のカテゴリー出現率、$ P(C) $：カテゴリー出現率、$ P(d|C) $：カテゴリー内の文書出現率
* スムージング：ゼロ頻度問題を緩和する手法
    * ゼロ頻度問題：学習データ内に出現しなかった単語は新規データで扱うことができないという問題のこと
* $ P(w_i|C)=\frac{C(w_i|C)+\alpha}{N_C+\alpha V} $
* $ C(w_i|C) $：単語$ w_i $のクラスCの文書中の出現頻度、$ N_C $：クラスCの文書の全単語数、$ V $：語彙数
* 全ての単語が実際よりごく少ない回数($ \alpha $回)出てきたと考えて、全ての単語に$ \alpha $を追加した式
* 参照：https://qiita.com/ishizakiiii/items/07cc7e463dceb3efe1a1
* とにかく、学習データを新規データに使えるように形にしたい！

* 単純グッド・チューリング法：$ r^* = (r+1)\frac{N_{r+1}}{N{r}} $  ($ N_r $：コーパス内でr回出てきた単語の数)

In [2]:
def Make_dict(file):   #学習データを読み込み、素性のインデックス辞書featsとラベルのインデックス辞書labelsを作成する関数
  feats, labels = {}, {}
  findex, lindex= 0, 0
  with open(file, encoding='utf-8') as f:
    for line in f:
      list=line.split(' ')  # 分かち書きリスト
      for item in list:
        word, right = item.split(":")
        #素性の処理
        if (word not in feats) and (word != "#label#"):
          feats[word]=findex
          findex+=1
        elif word == "#label#": #ラベルの処理
          right=right.replace('\n', '') 
          if right not in labels:
            labels[right]=lindex
            lindex+=1
    return feats, labels

In [3]:
features, labels = Make_dict('train.processed')  #Amazonレビューデータ

In [4]:
type(features)  #値はインデックス番号の辞書となっている

dict

In [5]:
len(features)  #素性数

19983

In [6]:
labels  #肯定・否定の２クラス

{'positive': 0, 'negative': 1}

In [7]:
def Make_sample_vectors(file, feats, label_dict):   #ファイルと、素性辞書とラベル辞書を引数にとり、用例ベクトルのリストとそれに対応する答えのリストを返す関数
  samples, label_list =[], []
  with open(file, encoding='utf-8') as f:
    for line in f:
      list=line.split(' ')
      asample = [0] * len(feats)
      for item in list:
        word, right = item.split(":")
        if word == "#label#": #ラベルの処理
          label_list.append(int(label_dict[right.replace('\n', '')])) # positive or negative = 0 or 1
        else:  #素性の処理
          if word in feats:
            asample[feats[word]]=int(right)
      samples.append(asample)
  return samples, label_list

In [8]:
train_X, train_Y = Make_sample_vectors('train.processed', features, labels)

In [9]:
test_X, test_Y = Make_sample_vectors('test.processed', features, labels)

# 多クラス分類の準備

In [15]:
import janome
import re
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
t = Tokenizer()
vectorizer = CountVectorizer(token_pattern='(?u)\\b\\w+\\b') #日本語対応


#ファイルを入れて分かち書きした文のリストと、それに対応するスコア（星の数）のリストを返す
def Get_Tokenized_senntences_and_Ratings(file): 

  with open(file, encoding='utf-8') as f:
    data = f.read()
    data = data.replace('\n', '')
    data = data.replace('\r', '')

  reviews = re.findall(pattern=r'<item>(.+?)</item>',string=data) #testにひとつだけratingはあるのにtextがない感想があったため
  
  tokenized_sentences, ratings = [],[]

  for index in range(len(reviews)):
    rating = re.findall(pattern=r'<rating>(.+?)</rating>',string=reviews[index])
    text = re.findall(pattern=r'<text>(.+?)</text>',string=reviews[index])
    if rating and text: #testにひとつだけtextがない感想があったため
      words=[token for token in t.tokenize(text[0], wakati=True)]
      ratings.append(int(float(rating[0]))-1)
      a_tokenized_sentence=" ".join(words)
      tokenized_sentences.append(a_tokenized_sentence)
 
  return tokenized_sentences, ratings

#ファイルを入れて分かち書きした文のリストと、それに対応するスコア（星の数）のリストを返す
def Make_sample_vectors_for_Multiclass(train_file, test_file):
  tokenized_sentences_train, ytrain =Get_Tokenized_senntences_and_Ratings(train_file)
  Xtrain = vectorizer.fit_transform(tokenized_sentences_train)
  feature_names = vectorizer.get_feature_names_out(tokenized_sentences_train)

  tokenized_sentences_test, ytest=Get_Tokenized_senntences_and_Ratings(test_file)
  Xtest = vectorizer.transform(tokenized_sentences_test)

  return Xtrain, ytrain, feature_names, Xtest, ytest

In [17]:
train_x, train_y, feat_names, test_x, test_y = Make_sample_vectors_for_Multiclass('train.review', 'test.review')

In [ ]:
type(train_X)

list

In [ ]:
len(train_X)

2000

In [ ]:
type(train_Y)

list

In [ ]:
len(train_Y)

2000

In [ ]:
type(train_X[0])

list

In [ ]:
len(train_X[0])

19983

In [20]:
from sklearn.naive_bayes import BernoulliNB
cl = BernoulliNB()
cl.fit(train_X, train_Y)

BernoulliNB()

In [ ]:
cl.score(test_X, test_Y)  #正解率

0.7485

In [ ]:
cl = BernoulliNB(alpha=0.5)  #加算スムージングのalphaを0.5に指定
cl.fit(train_X, train_Y)
cl.score(test_X, test_Y)

0.7565

In [ ]:
from sklearn.naive_bayes import MultinomialNB  #多クラス分類
cl2 = MultinomialNB()
cl2.fit(train_x, train_y)
cl2.score(test_x, test_y)

0.79

# 文書分類の評価

* 正解率が主流　正解数÷問題数
* スパムメールか否かを分類→「スパムメールをスパムメールを判断」と「スパムメールではないメールをスパムメールではないと判断」の２通りが正解数となる
* 精度、再現率、F値から評価する方がよい
* 精度：成り立つと予測したもののうち、実際に成り立つものの割合
* 再現率：実際に成り立つもののうち、成り立つと予測したものの割合
* 精度と再現率はトレードオフ→F値で評価
* $ F値=\frac{2×精度×再現率}{正解率+再現率} $

In [21]:
cl = BernoulliNB(alpha=1)  #加算スムージングのalphaを1に指定
cl.fit(train_X, train_Y)
cl.score(test_X, test_Y)

0.7485

In [22]:
test_ans_list = cl.predict(test_X)

In [ ]:
from sklearn.metrics import precision_score  #精度の計算
precision_score(test_Y, test_ans_list, average=None)

array([0.83001328, 0.69927827])

In [ ]:
test_ans_list2 = test_ans_list.tolist()  #ndarray型からリスト型二変換
test_ans_list2.count(0)  #肯定(0)に予測された数

753

In [ ]:
test_ans_list2.count(1)  #否定(1)に予測された数

1247

In [ ]:
[test_ans_list2[i]==0 and test_ans_list2[i] == test_Y[i] for i in range(len(test_ans_list2))].count(True)  #肯定の正解数

625

In [ ]:
[test_ans_list2[i]==1 and test_ans_list2[i] == test_Y[i] for i in range(len(test_ans_list2))].count(True)  #否定の正解数


872

In [ ]:
precision_score(test_Y, test_ans_list, average='micro')

0.7485

In [ ]:
precision_score(test_Y, test_ans_list, average='macro')

0.7646457740276531

In [ ]:
from sklearn.metrics import recall_score
recall_score(test_Y, test_ans_list, average=None)

array([0.625, 0.872])

In [ ]:
recall_score(test_Y, test_ans_list, average='micro')

0.7485

In [ ]:
recall_score(test_Y, test_ans_list, average='macro')

0.7484999999999999

In [23]:
from sklearn.metrics import f1_score
f1_score(test_Y, test_ans_list, average=None)

array([0.71306332, 0.77614597])

In [ ]:
f1_score(test_Y, test_ans_list, average='micro')

0.7485

In [ ]:
f1_score(test_Y, test_ans_list, average='macro')

0.7446046462152363

# ロジスティック回帰

* ロジスティック関数：0から1の間の出力となる
* 回帰の方法であるが、分類が可能である！
* 「出力が0.5以上の時は1とする」などとすれば、分類問題となる
* $ y=\frac{1}{1+e^{-ax}} $、xの次数によって決定境界の関数を決定
* $ x=\sum_{i=1}^{n}w_ix_i+b $
* $ w_i $を損失関数の最小化によって求める→尤度関数の最大化
* 損失関数：予測値と実測値のずれの大きさを表したもの→ロジスティック関数ならクロスエントロピー
* クロスエントロピー：2つの確率分布の間に定義される尺度
* $ Loss(w)=-\sum_{i=1}^{N}{(I_n\log(y_{n,w})+(1-I_n)\log(1-y_{n.w})}) $
* N：学習データ数、$ I_n $：n番目の学習データのラベル、$ y_{n,w} $：n番目の学習データの入力素性に対して、$ w_i $で学習した場合の予測確率

In [ ]:
from sklearn.linear_model import LogisticRegression
cl = LogisticRegression(random_state=0)

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train_X)  #単位の違いを調整するために標準化の変換器を作成
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
train_X_scaled[0]

array([ 2.00378077,  2.11182876,  1.48907797, ..., -0.02236627,
       -0.02236627, -0.02236627])

In [ ]:
test_X_scaled[0]

array([-0.78910723,  0.0687002 , -1.09694313, ..., -0.02236627,
       -0.02236627, -0.02236627])

In [ ]:
cl.fit(train_X_scaled, train_Y)

LogisticRegression(random_state=0)

In [ ]:
cl.score(test_X_scaled, test_Y)

0.7395

In [ ]:
test_ans_list = cl.predict(test_X_scaled)
precision_score(test_Y, test_ans_list, average='micro')

0.7395

In [24]:
f1_score(test_Y, test_ans_list, average='macro')

0.7446046462152363

* $ 正則化：Loss(w)=-C\sum_{i=1}^{N}{(I_n\log(y_{n,w})+(1-I_n)\log(1-y_{n.w})})+\frac{1}{2}\sum_{j=1}^{M}w_j^2 $
* C：正則化の程度を指定するオプション
* 重みパラメータの2乗を足し合わせることによって、重みパラメータの数を減らし、学習データに細かく合わせることができなくなるという点で、過学習を防ぐことができる。
* 2乗を足し合わせる→L2正則化、絶対値を足し合わせる→L1正則化

In [ ]:
# not標準化
cl2 = LogisticRegression(solver='liblinear')
cl2.fit(train_X, train_Y)
cl2.score(test_X, test_Y)

0.7875

In [ ]:
# 標準化
cl2.fit(train_X_scaled, train_Y)
cl2.score(test_X_scaled, test_Y)

0.738

### Naive Bayesの多クラス分類
* softmax関数：$ P(y_i|x)=\frac{e^{x_i}}{\sum_{k=1}^{K}e^{x_k}}$、K：クラスの数
* $ P(y_i|x) $：$ x_i $を入力にとったときにi番目のクラス$ y_i $が起こる確率
* $ e^x $は非負であるため、負の値から大きな値まで大小関係を保ったままの正の値に変換可能
* さらに、softmax関数では、その正の値を足して1になるように正規化することで、その出力を確率として扱うことができる
* $ K=2 $としたとき、ロジスティック関数と等価となる→ロジスティック関数を用いた多クラス分類への拡張がsoftmax関数の分類である
* クロスエントロピー：2つの確率分布の間に定義される尺度
* $ Loss(w)=-\sum_{n=1}^{N}\sum_{k=1}^{K}I_{n,k}\log(y_{n,w,k}) $

In [25]:
# 多クラス分類
from sklearn.linear_model import LogisticRegression
cl_soft = LogisticRegression(multi_class='multinomial', solver='newton-cg')
cl_soft.fit(train_x, train_y)
cl_soft.score(test_x, test_y)

0.4847423711855928

# サポートベクターマシン(SVM)

* 決定関数を学習で求めてから分類を行う→「超平面」
* 3次元空間上の点の集合を2次元の超平面で分類する
* 一般化すると、N次元空間上の点の集合はN-1次元の超平面で分類可能である
* サポートベクター：決定境界から最も近くにある用例
* サポートベクターと決定境界の距離マージンを最大化する決定関数を学習させる
* $ 決定関数：y=\sum_{i=1}^{n}w_ix_i+b $、$ x_i：一つの素性, w_i：重み, b：切片 $
* $ y=\bf{W}^T\bf{X}+b $
* 点(サポートベクター)と直線(決定関数)の距離：$ \frac{|\bf{W}^T\bf{X}+b|}{\sqrt{\bf{W}^2}} $ を最大化
* $ y=0 $(超平面H), $ y=1 $(超平面H1), $ y=-1 $(超平面H2)を考える
* $ y_i=\bf{W}^T X_j+b $ の値が1以上ならH1に、-1以下ならH2に分類する
* ソフトマージンについては、テキストを参照
* 二値分類を複数回実行して多クラス分類を行う→pairwise法とOne-vs-One法が存在
* pairwise法：クラスの対ごとにどちらのクラスに属するかを決定する手法
* A,B,Cの三つのクラスへの分類→A,B、B,C、C,Aの3回の二値分類を行う
* One-vs-One法：クラスごとにそのクラスに属するかを決定する手法
* A,B,Cの三つのクラスへの分類→Aか否か、Bか否か、Cか否かの3回の二値分類を行う
* どちらも矛盾が生じる場合があるが、分類平面との距離が遠い(距離マージンが大きくなる)方のクラスに決定すればよい

In [ ]:
from sklearn.svm import SVC
cl = SVC(gamma='auto')
cl.fit(train_X, train_Y)

SVC(gamma='auto')

In [ ]:
cl.score(test_X, test_Y)

0.5585

In [ ]:
cl_linear = SVC(gamma='auto', kernel='linear')
cl_linear.fit(train_X, train_Y)
cl_linear.score(test_X, test_Y)

0.7675

In [ ]:
# 多クラス分類
from sklearn.svm import SVC
cl_multi=SVC(gamma='auto', kernel='linear')
cl_multi.fit(train_x, train_y)
cl_multi.score(test_x, test_y)

0.45522761380690346

In [ ]:
cl_multi = SVC(gamma='auto', kernel='linear', decision_function_shape='ovo')  #One-vs-One法を用いる
cl_multi.fit(train_x, train_y)
cl_multi.score(test_x, test_y)

0.45522761380690346

In [ ]:
# polyカーネル
cl_multi_poly = SVC(gamma='auto', kernel='poly')
cl_multi_poly.fit(train_x, train_y)
cl_multi_poly.score(test_x, test_y)

0.34317158579289647

* LinearSVC

In [ ]:
from sklearn.svm import LinearSVC
cl = LinearSVC()
cl.fit(train_X, train_Y)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [ ]:
cl.score(test_X, test_Y)

0.768

In [ ]:
cl_1000 = LinearSVC(C=1000)  #正則化パラメータを変更
cl_1000.fit(train_X, train_Y)
cl_1000.score(test_X, test_Y)

0.767

In [27]:
from sklearn.svm import LinearSVC
cl = LinearSVC()
cl.fit(train_x, train_y)
cl.score(test_x, test_y)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.46473236618309155

In [28]:
cl = LinearSVC(C=1000)  #ハードマージンで分類
cl.fit(train_x, train_y)
cl.score(test_x, test_y)

0.45672836418209106

# ニューラルネットワーク

* $ y=f(\sum_{i=1}^{n}x_iw_i+b), y：出力, x_i：i番目の入力, w_i：x_iの重み, -b：閾値 $
* $ f $：活性化関数
* $ ステップ関数：f(I)=1,I≧0 ; f(I)=0,I＜0 $→入力が0を超えると、ニューロンが発火する(1になる)
* $ I=\sum_{i=1}^{n}x_iw_i+b $とすると、上記の式が得られる
* 一つのニューロンモデルからなるニューラルネットワーク→単純パーセプトロン

### 単純パーセプトロン
* ロジスティック関数→ロジスティック回帰式、損失関数→クロスエントロピー式
* softmax関数→多クラス分類のロジスティック回帰式、損失関数→多クラス分類のクロスエントロピー式
* 複数組み合わせたもの→多層パーセプトロン
* 多層パーセプトロンにおける入力層と出力層の間の層→中間層、隠れ層
* 中間層を増やすと、決定関数が複雑になり、線形ではない分け方が可能となる

### 誤差逆伝播法
* 参照(ヨビノリ)：https://www.youtube.com/watch?v=0itH0iDO8BE
* 革新的な重みの学習法
* 行列に対する勾配降下法
* 局所誤差の最小化

##### 手順
* ニューラルネットワークに学習サンプルを与える
* NNの出力から出力層における誤差を求め、各出力ニューロンについて誤差を計算する
* 個々のニューロンの期待される出力値と倍率、要求された出力と実際の出力との差(局所誤差)を計算する
* 各ニューロンの重みを局所誤差が小さくなるよう調整する
* より大きな重みで接続された前段のニューロンに対して、局所誤差の責任があると判定する
* そのように判定された前段のニューロンのさらに前段のニューロン群について同様の処理を行う

In [ ]:
from sklearn.neural_network import MLPClassifier
cl = MLPClassifier(hidden_layer_sizes=(3,))  
#hidden_layer_sizesで層のユニット数を指定、中間層１つで3つのユニットから構成されていることを表している、(3,4)とすればユニット数4の中間層を右に追加
cl.fit(train_X, train_Y)
cl.score(test_X, test_Y)

0.792

In [ ]:
# 多クラス分類
from sklearn.neural_network import MLPClassifier
cl = MLPClassifier(hidden_layer_sizes= (10,5))
cl.fit(train_x, train_y)
cl.score(test_x, test_y)

0.48624312156078037

# ディープラーニング

* 計算能力の向上により、中間層を増やすことが可能となった
* 勾配消失問題を解決するために発展した
* 勾配消失問題：損失関数を最小化するために偏微分を繰り返して勾配を求めているものの、勾配が0のときに損失関数が最小となる問題
* 勾配が0に近い場合、重みがほとんど更新されなくなる
* 誤差逆伝播法の場合も、勾配消失によって学習が進まなくなる
* $ Relu関数：f(I)=I,I≧0 ; f(I)=0,I＜0 $を活性化関数として学習を行う
* 入力が1以上の時は勾配が1となり、1をいくらかけても勾配は小さくならないため、勾配消失を防ぐことができる
* Batch Normalization、残差接続、重み共有、勾配クリッピング、ドロップアウトなどで、様々な問題を解決
* 畳み込みニューラルネットワーク(CNN)やリカレントニューラルネットワーク(RNN)などが発展
* 自然言語処理では、文という可変長のデータを取り扱うことから、RNNが広く使われている

# 半教師あり学習

* 正解を一部のデータにだけ与えて学習する方法
* 教師ありデータと教師なしデータの両方を与えて学習する
* 教師ありデータを用意する際に、アノテーションが必要となり、手間が増える
* アノテーション：コーパスに教師値の情報を追加すること
* 教師なしデータとともに用いることで、精度の向上と手間の削減のバランスをとる

* 教師ありデータを作成する手間を省きたい！！！

### 自己教示学習
* a.教師ありデータを利用して分類器のモデルを学習→b.そのモデルを用いて教師なしデータを分類→c.推定されたラベルを正解と見なして再び学習を行ってモデルを作成
* 分類器の精度が高いことを前提としている
* 2つの分類器を用いる共学習、3つの分類器を用いるトライトレーニングなどのバリエーションがある

### EMアルゴリズム
* 確率で割り振ってラベル付与
* 期待値を求める処理のEステップと、最大化を行うMステップを交互に行うことで性能を向上させていく手法

* 自己教示学習の実践

In [ ]:
features, labels = Make_dict('train.processed')
train_X, train_Y = Make_sample_vectors('train.processed', features, labels)
test_X, test_Y = Make_sample_vectors('test.processed', features, labels)

In [ ]:
import numpy as np
train_X_arr = np.array(train_X)  #list型からnp.ndarray型へ変換
train_Y_arr = np.array(train_Y)
test_X_arr = np.array(test_X)
test_Y_arr = np.array(test_Y)

In [ ]:
rng = np.random.RandomState(42)  #乱数発生のオブジェクト
random_unlabeled_points = rng.rand(train_Y_arr.shape[0]) < 0.3  
#randモジュールでtrain_Y_arrの要素数分の乱数を発生させ、発生させた乱数が0.3未満であればTrue、そうでなければFalseを返すブール値の配列を作成
train_Y_arr[random_unlabeled_points] = -1  #Trueなら-1に置換し、Falseならそのまま残す処理

In [ ]:
np.sum(train_Y_arr == -1)  #全2000件のうち、615件がラベルなしデータに分類された

615

In [ ]:
np.sum(train_Y_arr > -1)  #全2000件のうち、1385件がラベルありデータに分類された

1385

In [ ]:
train_Y_arr_new = train_Y_arr[~random_unlabeled_points]
train_X_arr_new = train_X_arr[~random_unlabeled_points]
train_Y_arr_new.shape  #ラベル付きのデータ1385件を学習データとする

(1385,)

In [ ]:
# 上記の学習データを用いてSVMを実行
from sklearn.svm import SVC
cl = SVC(probability=True, kernel='linear')  #自己教示学習を行う際に、推論の信頼性の予測値の確率が必要となるため、probability=Trueの指定が必要
cl.fit(train_X_arr_new, train_Y_arr_new)
cl.score(test_X_arr, test_Y_arr)

0.7645

In [ ]:
# 自己教示学習の実行
from sklearn.semi_supervised import SelfTrainingClassifier
svc = SVC(probability=True, kernel='linear')
self_training_model = SelfTrainingClassifier(svc)
self_training_model.fit(train_X_arr, train_Y_arr)
self_training_model.score(test_X_arr, test_Y_arr)

0.765

### ラベル伝播法
* グラフベースの半教師あり学習
* k近傍法カーネルを用いたラベル伝播法と重みづけを行った多数決によるk近傍法は類似している

### LabelPropagationモジュールとLabelSpreadingモジュールによるラベル伝播

In [ ]:
# a
import numpy as np
train_X_arr = np.array(train_X)
train_Y_arr = np.array(train_Y)
test_X_arr = np.array(test_X)
test_Y_arr = np.array(test_Y)
rng = np.random.RandomState(42)
random_unlabeled_points = rng.rand(train_Y_arr.shape[0]) < 0.3
labels = np.copy(train_Y_arr)  #train_y_arrのラベルを答え合わせに使うために複製
labels[random_unlabeled_points] = -1

In [ ]:
# b
from sklearn.semi_supervised import LabelPropagation
label_prop_model = LabelPropagation(kernel='knn', n_neighbors=3)
label_prop_model.fit(train_X_arr, labels)
label_prop_model.score(train_X_arr, train_Y_arr)

0.726

In [ ]:
# c
train_Y_arr2 = label_prop_model.predict(train_X_arr)  #ラベル伝播で推定したラベルを正解と見なす
from sklearn.svm import LinearSVC
cl = LinearSVC(C=1000)
cl.fit(train_X_arr, train_Y_arr2)
cl.score(test_X_arr, test_Y_arr)


0.642

In [ ]:
np.sum(labels > -1)  #教師ありデータの件数

1385

In [ ]:
# ラベルなしデータを利用せず、1385件の教師ありデータだけでSVMを実行
train_Y_arr_new = train_Y_arr[~random_unlabeled_points]
train_X_arr_new = train_X_arr[~random_unlabeled_points]
cl=LinearSVC(C=1000)
cl.fit(train_X_arr_new, train_Y_arr_new)
cl.score(test_X_arr, test_Y_arr)

0.7645

In [ ]:
# ラベル拡散法
from sklearn.semi_supervised import LabelSpreading
label_prop_model =  LabelSpreading(kernel='knn', n_neighbors=3)
label_prop_model.fit(train_X_arr, labels)
label_prop_model.score(train_X_arr, train_Y_arr)

0.717

In [ ]:
# ラベル伝播したラベルの結果を正解と見なしてSVMで学習
train_X_arr2=label_prop_model.predict(train_X_arr)
cl=LinearSVC(C=1000)
cl.fit(train_X_arr, train_Y_arr2)
cl.score(test_X_arr, test_Y_arr)

0.642

### ディープラーニングと共に使われる半教師あり学習の手法
* word2vecの使用→文書のクラスが分からなくても学習できるため、アノテーションされていないコーパスから学習できる
  * トランスダクティブ学習：半教師あり学習のもともと持っているラベルなしデータの予測だけを行い、新規データのラベルの予測をしない場合のこと
* fine-tuning：ニューラルネットワークの重みパラメータの初期値として別のモデルのために学習した重みパラメータを設定することにより、モデルの性能を上げる手法
* マルチタスク学習(?)：一部の重みを共有して学習する手法
* 敵対的学習：生成器と識別器という二つのネットワークを使用する手法
  * 生成器：できるだけ本物そっくりのサンプルを生成するモデル
  * 識別器：生成器で作った偽物のサンプルと、与えられた本物のサンプルを見分ける機能を持つモデル
  * 生成器と識別器を競わせるという点で、「敵対的」学習と呼ばれる
  * 敵対的生成ネットワーク(GAN)

### 能動学習
* 機械学習の最中に人手でラベル付けをして精度を上げる手法
* 教師ありデータでモデルを作成した後、教師なしデータを適用してみて、モデルが判断に困った用例について人手でラベル付けを行い、その新たにラベル付けしたデータをもともとあった教師ありデータに追加して学習を行うことを繰り返す
* SVMでは超平面から距離、ロジスティック回帰ではsoftmax関数やロジスティック関数の出力した確率からラベルを判断する
* ラベル付けを行う手間がかかるが、AIとの共同作業でラベル付けを行うため、効率よくラベル付けができる